# Construir un modelo de clasificación: Deliciosas cocinas asiáticas e indias


## Clasificadores de cocina 2

En esta segunda lección sobre clasificación, exploraremos `más formas` de clasificar datos categóricos. También aprenderemos sobre las implicaciones de elegir un clasificador sobre otro.

### [**Cuestionario previo a la lección**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Requisito previo**

Asumimos que has completado las lecciones anteriores, ya que retomaremos algunos conceptos que aprendimos antes.

Para esta lección, necesitaremos los siguientes paquetes:

-   `tidyverse`: El [tidyverse](https://www.tidyverse.org/) es una [colección de paquetes de R](https://www.tidyverse.org/packages) diseñada para hacer la ciencia de datos más rápida, fácil y divertida.

-   `tidymodels`: El marco de trabajo [tidymodels](https://www.tidymodels.org/) es una [colección de paquetes](https://www.tidymodels.org/packages/) para modelado y aprendizaje automático.

-   `themis`: El paquete [themis](https://themis.tidymodels.org/) proporciona pasos adicionales de recetas para tratar con datos desbalanceados.

Puedes instalarlos con el siguiente comando:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativamente, el siguiente script verifica si tienes los paquetes necesarios para completar este módulo y los instala por ti en caso de que falten.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Un mapa de clasificación**

En nuestra [lección anterior](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), intentamos abordar la pregunta: ¿cómo elegimos entre múltiples modelos? En gran medida, depende de las características de los datos y del tipo de problema que queremos resolver (por ejemplo, ¿clasificación o regresión?).

Anteriormente, aprendimos sobre las diversas opciones que tienes al clasificar datos utilizando la hoja de referencia de Microsoft. El marco de aprendizaje automático de Python, Scikit-learn, ofrece una hoja de referencia similar pero más detallada que puede ayudarte aún más a reducir tus estimadores (otro término para clasificadores):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Consejo: [visita este mapa en línea](https://scikit-learn.org/stable/tutorial/machine_learning_map/) y haz clic a lo largo del camino para leer la documentación.  
>  
> El [sitio de referencia de Tidymodels](https://www.tidymodels.org/find/parsnip/#models) también ofrece una excelente documentación sobre los diferentes tipos de modelos.

### **El plan** 🗺️

Este mapa es muy útil una vez que tienes un entendimiento claro de tus datos, ya que puedes 'caminar' por sus caminos hacia una decisión:

-   Tenemos \>50 muestras

-   Queremos predecir una categoría

-   Tenemos datos etiquetados

-   Tenemos menos de 100K muestras

-   ✨ Podemos elegir un Linear SVC

-   Si eso no funciona, dado que tenemos datos numéricos

    -   Podemos intentar un ✨ KNeighbors Classifier

        -   Si eso tampoco funciona, probar ✨ SVC y ✨ Ensemble Classifiers

Este es un camino muy útil a seguir. Ahora, vamos a sumergirnos directamente utilizando el marco de modelado [tidymodels](https://www.tidymodels.org/): una colección consistente y flexible de paquetes de R desarrollada para fomentar buenas prácticas estadísticas 😊.

## 2. Dividir los datos y manejar un conjunto de datos desequilibrado.

En nuestras lecciones anteriores, aprendimos que había un conjunto de ingredientes comunes entre nuestras cocinas. Además, había una distribución bastante desigual en el número de cocinas.

Abordaremos esto de la siguiente manera:

-   Eliminando los ingredientes más comunes que generan confusión entre cocinas distintas, usando `dplyr::select()`.

-   Usando una `recipe` que preprocesa los datos para prepararlos para el modelado aplicando un algoritmo de `over-sampling`.

Ya vimos lo anterior en la lección pasada, ¡así que esto será pan comido 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Cómo manejar datos desbalanceados

Los datos desbalanceados a menudo tienen efectos negativos en el rendimiento del modelo. Muchos modelos funcionan mejor cuando el número de observaciones es igual y, por lo tanto, tienden a tener dificultades con datos desbalanceados.

Existen principalmente dos formas de abordar conjuntos de datos desbalanceados:

-   agregar observaciones a la clase minoritaria: `Sobre-muestreo`, por ejemplo, utilizando un algoritmo SMOTE que genera de manera sintética nuevos ejemplos de la clase minoritaria utilizando los vecinos más cercanos de estos casos.

-   eliminar observaciones de la clase mayoritaria: `Sub-muestreo`

En nuestra lección anterior, demostramos cómo manejar conjuntos de datos desbalanceados utilizando una `receta`. Una receta puede considerarse como un plan que describe qué pasos deben aplicarse a un conjunto de datos para prepararlo para el análisis. En nuestro caso, queremos tener una distribución equitativa en el número de nuestras categorías de cocina para nuestro `conjunto de entrenamiento`. Vamos a ello.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

¡Ahora estamos listos para entrenar modelos 👩‍💻👨‍💻!

## 3. Más allá de los modelos de regresión multinomial

En nuestra lección anterior, vimos los modelos de regresión multinomial. Ahora exploremos algunos modelos más flexibles para clasificación.

### Máquinas de Vectores de Soporte

En el contexto de la clasificación, las `Máquinas de Vectores de Soporte` son una técnica de aprendizaje automático que intenta encontrar un *hiperplano* que "mejor" separe las clases. Veamos un ejemplo sencillo:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ no separa las clases. H2~ sí lo hace, pero solo con un pequeño margen. H3~ las separa con el margen máximo.

#### Clasificador Lineal de Vectores de Soporte

El clustering de vectores de soporte (SVC) es una técnica derivada de la familia de máquinas de vectores de soporte (SVM) en el aprendizaje automático. En SVC, el hiperplano se elige para separar correctamente a `la mayoría` de las observaciones de entrenamiento, pero `puede clasificar erróneamente` algunas observaciones. Al permitir que algunos puntos estén en el lado incorrecto, el SVM se vuelve más robusto frente a valores atípicos, lo que mejora su capacidad de generalización a nuevos datos. El parámetro que regula esta violación se denomina `coste`, y tiene un valor predeterminado de 1 (consulta `help("svm_poly")`).

Vamos a crear un SVC lineal configurando `degree = 1` en un modelo SVM polinómico.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Ahora que hemos capturado los pasos de preprocesamiento y la especificación del modelo en un *workflow*, podemos proceder a entrenar el SVC lineal y evaluar los resultados al mismo tiempo. Para las métricas de rendimiento, vamos a crear un conjunto de métricas que evalúe: `accuracy`, `sensitivity`, `Positive Predicted Value` y `F Measure`.

> `augment()` añadirá columna(s) con las predicciones a los datos proporcionados.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Máquina de Vectores de Soporte

La máquina de vectores de soporte (SVM, por sus siglas en inglés) es una extensión del clasificador de vectores de soporte diseñada para manejar límites no lineales entre las clases. En esencia, las SVM utilizan el *truco del kernel* para ampliar el espacio de características y adaptarse a relaciones no lineales entre las clases. Una función kernel popular y extremadamente flexible que utilizan las SVM es la *función de base radial.* Veamos cómo se desempeña con nuestros datos.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

¡Mucho mejor 🤩!

> ✅ Por favor consulta:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> para más información.

### Clasificadores de Vecinos Más Cercanos

El algoritmo de *K*-vecinos más cercanos (KNN) predice cada observación basándose en su *similitud* con otras observaciones.

Vamos a ajustarlo a nuestros datos.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Parece que este modelo no está funcionando tan bien. Probablemente cambiar los argumentos del modelo (consulta `help("nearest_neighbor")`) mejorará el rendimiento del modelo. Asegúrate de probarlo.

> ✅ Por favor consulta:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> para aprender más sobre los clasificadores *K*-Nearest Neighbors.

### Clasificadores en conjunto

Los algoritmos en conjunto funcionan combinando múltiples estimadores base para producir un modelo óptimo, ya sea mediante:

`bagging`: aplicando una *función de promediado* a una colección de modelos base

`boosting`: construyendo una secuencia de modelos que se basan unos en otros para mejorar el rendimiento predictivo.

Comencemos probando un modelo de Random Forest, que construye una gran colección de árboles de decisión y luego aplica una función de promediado para obtener un modelo general mejorado.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

¡Buen trabajo 👏!

Vamos a experimentar también con un modelo de Árbol Potenciado.

El Árbol Potenciado define un método de conjunto que crea una serie de árboles de decisión secuenciales, donde cada árbol depende de los resultados de los árboles anteriores en un intento de reducir el error de manera incremental. Se enfoca en los pesos de los elementos clasificados incorrectamente y ajusta el modelo del siguiente clasificador para corregirlos.

Existen diferentes formas de ajustar este modelo (consulta `help("boost_tree")`). En este ejemplo, ajustaremos Árboles Potenciados utilizando el motor `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Por favor consulta:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Explora el modelo AdaBoost, que es una buena alternativa a xgboost.
>
> para aprender más sobre clasificadores Ensemble.

## 4. Extra - comparando múltiples modelos

Hemos ajustado bastantes modelos en este laboratorio 🙌. Puede volverse tedioso o complicado crear muchos flujos de trabajo a partir de diferentes conjuntos de preprocesadores y/o especificaciones de modelos y luego calcular las métricas de rendimiento una por una.

Veamos si podemos abordar esto creando una función que ajuste una lista de flujos de trabajo en el conjunto de entrenamiento y luego devuelva las métricas de rendimiento basadas en el conjunto de prueba. Usaremos `map()` y `map_dfr()` del paquete [purrr](https://purrr.tidyverse.org/) para aplicar funciones a cada elemento de una lista.

> Las funciones [`map()`](https://purrr.tidyverse.org/reference/map.html) te permiten reemplazar muchos bucles for con un código que es más conciso y fácil de leer. El mejor lugar para aprender sobre las funciones [`map()`](https://purrr.tidyverse.org/reference/map.html) es el [capítulo de iteración](http://r4ds.had.co.nz/iteration.html) en R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


El paquete [**workflowset**](https://workflowsets.tidymodels.org/) permite a los usuarios crear y ajustar fácilmente una gran cantidad de modelos, pero está diseñado principalmente para trabajar con técnicas de remuestreo como `cross-validation`, un enfoque que aún no hemos cubierto.

## **🚀Desafío**

Cada una de estas técnicas tiene una gran cantidad de parámetros que puedes ajustar, por ejemplo, `cost` en SVMs, `neighbors` en KNN, `mtry` (Predictores Seleccionados Aleatoriamente) en Random Forest.

Investiga los parámetros predeterminados de cada uno y piensa en lo que significaría ajustar estos parámetros para la calidad del modelo.

Para obtener más información sobre un modelo en particular y sus parámetros, utiliza: `help("model")`, por ejemplo, `help("rand_forest")`.

> En la práctica, usualmente *estimamos* los *mejores valores* para estos entrenando muchos modelos en un `conjunto de datos simulado` y midiendo qué tan bien funcionan todos estos modelos. Este proceso se llama **ajuste**.

### [**Cuestionario post-clase**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Revisión y Estudio Personal**

Hay mucho vocabulario técnico en estas lecciones, así que tómate un momento para revisar [esta lista](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) de terminología útil.

#### GRACIAS A:

[`Allison Horst`](https://twitter.com/allison_horst/) por crear las increíbles ilustraciones que hacen que R sea más acogedor y atractivo. Encuentra más ilustraciones en su [galería](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) y [Jen Looper](https://www.twitter.com/jenlooper) por crear la versión original en Python de este módulo ♥️

Feliz aprendizaje,

[Eric](https://twitter.com/ericntay), Embajador Estudiantil de Microsoft Learn Gold.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Arte por @allison_horst</figcaption>



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
